# Running the experiments for all models.

Different models need different experiments to characterize them, I may automate these decisions, but it's worth doing it manually at least once. 

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.core.pylabtools import figsize

import seaborn as sns

import numpy as np
import pandas as pd

from tqdm import tqdm

## Download dataset split

In [2]:
from initialization.download_datasubset import download_stratified_image_subsets 
base_dataset = "ILSVRC/imagenet-1k"

download_stratified_image_subsets(base_dataset, splits=['train', 'validation'], split_mapping={'validation': 'test'})


Resolving data files:   0%|          | 0/294 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/28 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/294 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/28 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/46 [00:00<?, ?it/s]

Saving the dataset (0/46 shards):   0%|          | 0/200000 [00:00<?, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Saving the dataset (0/14 shards):   0%|          | 0/50000 [00:00<?, ? examples/s]

## Experiments:

In [2]:
from setup import image_model_setup
from initialization.activity_dataset import generate_activity_dataset
from analysis import run_across_layers, apply_model_decoder, accuracy_random_CLS,\
                     linear_probe_by_ridge_regression, model_accuracy
from training import fit_probes_by_ridge_regression

W0122 09:12:12.428000 39512 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


import error: No module named 'triton'


### google/vit-base-patch16-224

The baseline case

In [5]:
model_name = "google/vit-base-patch16-224"
dataset_name = "temp_dataset_subsample"


In [6]:
model, image_datasets, _ = image_model_setup(model_name, dataset_name, 1000)

Resolving data files:   0%|          | 0/47 [00:00<?, ?it/s]

In [7]:
activity_dataset = generate_activity_dataset(model, image_datasets,
                                             include_classifier_inputs=True,
                                             output_dir='temp_activity_dataset',
                                             device='mps'
                                            )

Generating train split: 0 examples [00:00, ? examples/s]

W0121 10:44:56.923000 6100 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0121 10:44:56.923000 6098 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0121 10:44:56.931000 6101 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0121 10:44:56.931000 6099 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
/Users/luke/.local/defaultPythonEnv/lib/python3.12/site-packages/PIL/TiffImagePlugin.py:870: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))
/Users/luke/.local/defaultPythonEnv/lib/python3.12/site-packages/PIL/TiffImagePlugin.py:870: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


import error: No module named 'triton'
import error: No module named 'triton'
import error: No module named 'triton'
import error: No module named 'triton'


Generating test split: 0 examples [00:00, ? examples/s]

W0121 11:05:36.730000 29862 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0121 11:05:36.730000 29863 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0121 11:05:36.730000 29861 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0121 11:05:36.730000 29864 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


import error: No module named 'triton'
import error: No module named 'triton'
import error: No module named 'triton'
import error: No module named 'triton'


Saving the dataset (0/16 shards):   0%|          | 0/200000 [00:00<?, ? examples/s]

Saving the dataset (0/4 shards):   0%|          | 0/50000 [00:00<?, ? examples/s]

In [8]:
acc = model_accuracy(model.model, image_datasets['test'], shuffle=None, device='mps', num_workers=4)

0it [00:00, ?it/s]W0121 11:11:01.032000 29911 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0121 11:11:01.032000 29912 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0121 11:11:01.032000 29910 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0121 11:11:01.032000 29913 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
338it [04:08,  1.58it/s]

import error: No module named 'triton'
import error: No module named 'triton'


392it [04:46,  1.86it/s]

import error: No module named 'triton'
import error: No module named 'triton'


392it [04:47,  1.36it/s]


In [9]:
acc

0.8027200102806091

High accuracy to start with: no probe training needed.

In [10]:
all_results = []
probe_results = linear_probe_by_ridge_regression(activity_dataset, cvfold=5)
all_results.append(probe_results)

In [11]:
decoder_results = apply_model_decoder(model, activity_dataset)
all_results.append(decoder_results)

In [14]:
randomization_results = run_across_layers(model, image_datasets,
                                          accuracy_random_CLS,  shuffle=None,
                                          device='mps')
all_results.append(randomization_results)

📕 Analysis: <function accuracy_random_CLS at 0x31ba2e520>
📄 Layer: vit.encoder.layer.0


391it [07:04,  1.09s/it]


📄 Layer: vit.encoder.layer.1


391it [06:59,  1.07s/it]


📄 Layer: vit.encoder.layer.2


...

391it [06:55,  1.06s/it]


📄 Layer: vit.encoder.layer.10


391it [06:53,  1.06s/it]


📄 Layer: vit.encoder.layer.11


391it [06:53,  1.06s/it]


In [17]:
pd.concat(all_results).to_csv('results/cross_model_CLS_2/results_vit.csv')

### facebook/deit-base-distilled-patch16-224

Requires analysis with and without distillation tokens.


In [3]:
model_name = "facebook/deit-base-distilled-patch16-224"
dataset_name = "temp_dataset_subsample"

model, image_datasets, _ = image_model_setup(model_name, dataset_name, 1000)



Resolving data files:   0%|          | 0/47 [00:00<?, ?it/s]

In [4]:
activity_dataset = generate_activity_dataset(model, image_datasets,
                                             include_classifier_inputs=True,
                                             output_dir='temp_activity_dataset',
                                             device='mps'
                                            )

Generating train split: 0 examples [00:00, ? examples/s]

W0121 15:13:29.775000 31889 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0121 15:13:29.798000 31888 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0121 15:13:29.812000 31890 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0121 15:13:29.830000 31891 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
/Users/luke/.local/defaultPythonEnv/lib/python3.12/site-packages/PIL/TiffImagePlugin.py:870: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))
/Users/luke/.local/defaultPythonEnv/lib/python3.12/site-packages/PIL/TiffImagePlugin.py:870: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


import error: No module named 'triton'
import error: No module named 'triton'
import error: No module named 'triton'
import error: No module named 'triton'


Generating test split: 0 examples [00:00, ? examples/s]

W0121 15:33:44.797000 32100 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0121 15:33:44.797000 32101 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0121 15:33:44.797000 32103 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0121 15:33:44.798000 32102 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


import error: No module named 'triton'
import error: No module named 'triton'
import error: No module named 'triton'
import error: No module named 'triton'


Saving the dataset (0/16 shards):   0%|          | 0/200000 [00:00<?, ? examples/s]

Saving the dataset (0/4 shards):   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
# double check accuracy and operation of the hook

acc = model_accuracy(model.model, image_datasets['test'], shuffle=None, device='mps', num_workers=4)

with model.zero_out_auxiliary_outputs():
    acc_no_distillation = model_accuracy(model.model, image_datasets['test'], shuffle=None, device='mps', num_workers=4)

acc_post = model_accuracy(model.model, image_datasets['test'], shuffle=None, device='mps', num_workers=4)

0it [00:00, ?it/s]W0121 14:52:38.619000 31689 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0121 14:52:38.622000 31688 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0121 14:52:38.796000 31690 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0121 14:52:38.804000 31691 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
338it [03:51,  1.71it/s]

import error: No module named 'triton'
import error: No module named 'triton'


392it [04:31,  2.00it/s]

import error: No module named 'triton'
import error: No module named 'triton'


392it [04:32,  1.44it/s]
0it [00:00, ?it/s]W0121 14:57:11.400000 31716 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0121 14:57:11.400000 31718 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0121 14:57:11.400000 31717 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0121 14:57:11.400000 31719 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
338it [04:00,  1.55it/s]

...

392it [04:41,  1.39it/s]
0it [00:00, ?it/s]W0121 15:01:54.235000 31760 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0121 15:01:54.235000 31761 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0121 15:01:54.235000 31762 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0121 15:01:54.235000 31759 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
338it [04:11,  1.43it/s]

import error: No module named 'triton'
import error: No module named 'triton'


392it [04:53,  1.73it/s]

import error: No module named 'triton'
import error: No module named 'triton'


392it [04:54,  1.33it/s]


In [5]:
print(acc, acc_no_distillation, acc_post)

0.83406001329422 0.8289600014686584 0.83406001329422


Looks good: zeroing out distillation tokens has an effect, but only a small one.

In [5]:
# CLS token probing is un-effected by distillation outputs
all_results = []
probe_results = linear_probe_by_ridge_regression(activity_dataset, cvfold=5)
all_results.append(probe_results)

In [6]:
randomization_results = run_across_layers(model, image_datasets,
                                          accuracy_random_CLS,  shuffle=None,
                                          device='mps')
all_results.append(randomization_results)

📕 Analysis: <function accuracy_random_CLS at 0x324bb9d00>
📄 Layer: deit.encoder.layer.0


391it [07:03,  1.08s/it]


📄 Layer: deit.encoder.layer.1


391it [06:55,  1.06s/it]


📄 Layer: deit.encoder.layer.2


...

391it [06:52,  1.06s/it]


📄 Layer: deit.encoder.layer.10


391it [06:53,  1.06s/it]


📄 Layer: deit.encoder.layer.11


391it [06:57,  1.07s/it]


In [7]:
with model.zero_out_auxiliary_outputs():
    randomization_results = run_across_layers(model, image_datasets,
                                              accuracy_random_CLS,  shuffle=None,
                                              device='mps')

randomization_results['distillation_disabled'] = True
all_results.append(randomization_results)

📕 Analysis: <function accuracy_random_CLS at 0x324bb9d00>
📄 Layer: deit.encoder.layer.0


391it [07:00,  1.08s/it]


📄 Layer: deit.encoder.layer.1


391it [06:57,  1.07s/it]


📄 Layer: deit.encoder.layer.2


...

391it [06:58,  1.07s/it]


📄 Layer: deit.encoder.layer.10


391it [06:58,  1.07s/it]


📄 Layer: deit.encoder.layer.11


391it [06:58,  1.07s/it]


In [9]:
pd.concat(all_results).to_csv('results/cross_model_CLS/results_deit.csv')

### facebook/dinov2-base

Requires probe training,  analysis with and without GAP outputs

In [3]:
model_name = "facebook/dinov2-base"
dataset_name = "temp_dataset_subsample"

model, image_datasets, _ = image_model_setup(model_name, dataset_name, 1000)


Resolving data files:   0%|          | 0/47 [00:00<?, ?it/s]

Some weights of Dinov2ForImageClassification were not initialized from the model checkpoint at facebook/dinov2-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
activity_dataset = generate_activity_dataset(model, image_datasets,
                                             include_classifier_inputs=True,
                                             output_dir='temp_activity_dataset',
                                             device='mps'
                                            )

Generating train split: 0 examples [00:00, ? examples/s]

W0121 21:35:18.754000 34520 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0121 21:35:18.820000 34521 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0121 21:35:18.965000 34522 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0121 21:35:18.966000 34523 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
/Users/luke/.local/defaultPythonEnv/lib/python3.12/site-packages/PIL/TiffImagePlugin.py:870: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))
/Users/luke/.local/defaultPythonEnv/lib/python3.12/site-packages/PIL/TiffImagePlugin.py:870: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


import error: No module named 'triton'
import error: No module named 'triton'
import error: No module named 'triton'
import error: No module named 'triton'


Loading dataset shards:   0%|          | 0/17 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

W0121 22:03:03.825000 34848 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0121 22:03:03.825000 34846 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0121 22:03:03.825000 34847 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0121 22:03:03.825000 34849 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


import error: No module named 'triton'
import error: No module named 'triton'
import error: No module named 'triton'
import error: No module named 'triton'


Saving the dataset (0/18 shards):   0%|          | 0/200000 [00:00<?, ? examples/s]

Saving the dataset (0/5 shards):   0%|          | 0/50000 [00:00<?, ? examples/s]

Loading dataset from disk:   0%|          | 0/18 [00:00<?, ?it/s]

In [5]:
fit_probes_by_ridge_regression(model, activity_dataset)

In [6]:
acc = model_accuracy(model.model, image_datasets['test'], shuffle=None, device='mps', num_workers=4)

with model.zero_out_auxiliary_outputs():
    acc_no_GAP = model_accuracy(model.model, image_datasets['test'], shuffle=None, device='mps', num_workers=4)

acc_post = model_accuracy(model.model, image_datasets['test'], shuffle=None, device='mps', num_workers=4)

0it [00:00, ?it/s]W0121 22:14:21.476000 35035 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0121 22:14:21.476000 35032 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0121 22:14:21.476000 35034 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0121 22:14:21.476000 35033 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
338it [06:13,  1.13s/it]

import error: No module named 'triton'
import error: No module named 'triton'


392it [07:14,  1.16it/s]

import error: No module named 'triton'
import error: No module named 'triton'


392it [07:15,  1.11s/it]
0it [00:00, ?it/s]W0121 22:21:37.080000 35151 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0121 22:21:37.080000 35153 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0121 22:21:37.081000 35154 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0121 22:21:37.082000 35152 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
338it [06:27,  1.11s/it]

...

392it [07:28,  1.15s/it]
0it [00:00, ?it/s]W0121 22:29:05.838000 35256 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0121 22:29:05.860000 35253 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0121 22:29:05.866000 35255 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0121 22:29:05.867000 35254 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
338it [06:31,  1.11s/it]

import error: No module named 'triton'
import error: No module named 'triton'


392it [07:32,  1.12it/s]

import error: No module named 'triton'
import error: No module named 'triton'


392it [07:33,  1.16s/it]


In [7]:
print(acc, acc_no_GAP, acc_post)

0.7628399729728699 0.7245200276374817 0.7628399729728699


In [8]:
all_results = []
probe_results = linear_probe_by_ridge_regression(activity_dataset, cvfold=5)
all_results.append(probe_results)

In [9]:
randomization_results = run_across_layers(model, image_datasets,
                                          accuracy_random_CLS,  shuffle=None,
                                          device='mps')
all_results.append(randomization_results)

📕 Analysis: <function accuracy_random_CLS at 0x33cbb9da0>
📄 Layer: dinov2.encoder.layer.0


391it [09:51,  1.51s/it]


📄 Layer: dinov2.encoder.layer.1


391it [09:27,  1.45s/it]


📄 Layer: dinov2.encoder.layer.2


...

391it [09:15,  1.42s/it]


📄 Layer: dinov2.encoder.layer.10


391it [09:16,  1.42s/it]


📄 Layer: dinov2.encoder.layer.11


391it [09:15,  1.42s/it]


In [10]:
with model.zero_out_auxiliary_outputs():
    randomization_results = run_across_layers(model, image_datasets,
                                              accuracy_random_CLS,  shuffle=None,
                                              device='mps')

randomization_results['GAP_disabled'] = True
all_results.append(randomization_results)

📕 Analysis: <function accuracy_random_CLS at 0x33cbb9da0>
📄 Layer: dinov2.encoder.layer.0


391it [09:15,  1.42s/it]


📄 Layer: dinov2.encoder.layer.1


391it [09:16,  1.42s/it]


📄 Layer: dinov2.encoder.layer.2


...

391it [09:16,  1.42s/it]


📄 Layer: dinov2.encoder.layer.10


391it [09:16,  1.42s/it]


📄 Layer: dinov2.encoder.layer.11


391it [09:16,  1.42s/it]


In [12]:
pd.concat(all_results).to_csv('results/cross_model_CLS/results_dino.csv')

### Beit

We need to properly configure the model to use CLS tokens rather than mean pooling

In [3]:
model_name = "microsoft/beit-base-patch16-224-pt22k-ft22k"
dataset_name = "temp_dataset_subsample"

model, image_datasets, _ = image_model_setup(model_name, dataset_name, 1000)

Resolving data files:   0%|          | 0/47 [00:00<?, ?it/s]

Some weights of the model checkpoint at microsoft/beit-base-patch16-224-pt22k-ft22k were not used when initializing BeitForImageClassification: ['beit.pooler.layernorm.bias', 'beit.pooler.layernorm.weight']
- This IS expected if you are initializing BeitForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BeitForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BeitForImageClassification were not initialized from the model checkpoint at microsoft/beit-base-patch16-224-pt22k-ft22k and are newly initialized: ['beit.layernorm.bias', 'beit.layernorm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictio

In [4]:
activity_dataset = generate_activity_dataset(model, image_datasets,
                                             include_classifier_inputs=True,
                                             output_dir='temp_activity_dataset',
                                             device='mps'
                                            )

Generating train split: 0 examples [00:00, ? examples/s]

W0122 09:12:54.079000 39518 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0122 09:12:54.084000 39517 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0122 09:12:54.087000 39520 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0122 09:12:54.101000 39519 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
/Users/luke/.local/defaultPythonEnv/lib/python3.12/site-packages/PIL/TiffImagePlugin.py:870: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))
/Users/luke/.local/defaultPythonEnv/lib/python3.12/site-packages/PIL/TiffImagePlugin.py:870: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


import error: No module named 'triton'
import error: No module named 'triton'
import error: No module named 'triton'
import error: No module named 'triton'


Generating test split: 0 examples [00:00, ? examples/s]

W0122 09:37:20.784000 39793 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0122 09:37:20.784000 39791 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0122 09:37:20.784000 39792 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0122 09:37:20.784000 39794 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


import error: No module named 'triton'
import error: No module named 'triton'
import error: No module named 'triton'
import error: No module named 'triton'


Saving the dataset (0/16 shards):   0%|          | 0/200000 [00:00<?, ? examples/s]

Saving the dataset (0/4 shards):   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
fit_probes_by_ridge_regression(model, activity_dataset)

In [6]:
acc = model_accuracy(model.model, image_datasets['test'], shuffle=None, device='mps', num_workers=4)


0it [00:00, ?it/s]W0122 09:46:44.251000 39893 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0122 09:46:44.259000 39892 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0122 09:46:44.264000 39894 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0122 09:46:44.264000 39891 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
338it [04:52,  1.22it/s]

import error: No module named 'triton'
import error: No module named 'triton'


392it [05:38,  1.45it/s]

import error: No module named 'triton'
import error: No module named 'triton'


392it [05:39,  1.16it/s]


In [7]:
print(acc)

0.7053999900817871


Good enough.

In [8]:
all_results = []
probe_results = linear_probe_by_ridge_regression(activity_dataset, cvfold=5)
all_results.append(probe_results)

In [9]:
randomization_results = run_across_layers(model, image_datasets,
                                          accuracy_random_CLS,  shuffle=None,
                                          device='mps')
all_results.append(randomization_results)

📕 Analysis: <function accuracy_random_CLS at 0x31aab5f80>
📄 Layer: beit.encoder.layer.0


391it [07:46,  1.19s/it]


📄 Layer: beit.encoder.layer.1


391it [07:25,  1.14s/it]


📄 Layer: beit.encoder.layer.2


...

391it [07:25,  1.14s/it]


📄 Layer: beit.encoder.layer.10


391it [07:24,  1.14s/it]


📄 Layer: beit.encoder.layer.11


391it [07:24,  1.14s/it]


In [10]:
all_results

[                    layer  accuracy             test
 0    beit.encoder.layer.0   0.01224  layerwise_probe
 1    beit.encoder.layer.1   0.08510  layerwise_probe
 2    beit.encoder.layer.2   0.11900  layerwise_probe
 3    beit.encoder.layer.3   0.15024  layerwise_probe
 4    beit.encoder.layer.4   0.20848  layerwise_probe
 5    beit.encoder.layer.5   0.23442  layerwise_probe
 6    beit.encoder.layer.6   0.30842  layerwise_probe
 7    beit.encoder.layer.7   0.37152  layerwise_probe
 8    beit.encoder.layer.8   0.42894  layerwise_probe
 9    beit.encoder.layer.9   0.49438  layerwise_probe
 10  beit.encoder.layer.10   0.56520  layerwise_probe
 11  beit.encoder.layer.11   0.71438  layerwise_probe
 12      classifier_inputs   0.70898  layerwise_probe,
                     layer  accuracy               test
 0    beit.encoder.layer.0   0.70554  cls_randomization
 1    beit.encoder.layer.1   0.70510  cls_randomization
 2    beit.encoder.layer.2   0.70504  cls_randomization
 3    beit.encoder.

In [11]:
pd.concat(all_results).to_csv('results/cross_model_CLS/results_beit.csv')

### openai/clip-vit-base-patch32

The previous results are sufficient for now.
